In [ ]:
import pandas as pd

In [ ]:
housing = pd.read_csv('cal_housing_clean.csv')

housing.head()

In [ ]:
housing.describe().transpose()

In [ ]:
# separating label i.e. y_value
# and x_data

In [ ]:
y_val = housing["medianHouseValue"]
x_data = housing.drop("medianHouseValue", axis = 1)


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_val, test_size= 0.33, random_state = 101)

In [5]:
from sklearn.preprocessing import MinMaxScaler
# scaling i.e normalziing the data

scaler = MinMaxScaler()  # initiazlizing min max scaller
scaler.fit(x_train)   # we only fit that on training data

In [ ]:
x_train = pd.DataFrame(data = scaler.transform(x_train),
                       columns = x_train.columns,
                       index = x_train.index)

x_test = pd.DataFrame(data = scaler.transform(x_test),
                       columns = x_test.columns,
                       index = x_test.index)

In [ ]:
scaler.transform(x_train)

In [ ]:
# now we r gonna create feature columns 
# create the necessary tf.feature_column object for estimator. They should all be treated as continous numeric_columns. 
housing.columns 

In [ ]:
import tensorflow as tf

# is it is numeric column then we will use this way.
# notice they all are numeric values
age = tf.feature_column.numeric_column("housingMedianAge")
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')


In [ ]:
feat_cols = [age, rooms, bedrooms, pop, households, income]

input_func = tf.estimator.inputs.pandas_input_fn(x= x_train, y = y_train, batch_size= 10, num_epochs = 1000, shuffle = True)
# this is our trainig function, we will pass this in our estimtor api

In [ ]:
# Create the estimator model. Use a DNNREgressor. Play around with the hidden units 
    
model = tf.estimator.DNNRegressor(hidden_units = [6, 6, 6], feature_columns = feat_cols)
    
model.train(input_fn = input_func, steps = 20000)

In [ ]:
# now i m testing my model with the test data. I can create my own random data and test it here. 
predict_input_func = tf.estimator.inputs.pandas_input_fn(x = x_test, batch_size = 10, num_epochs = 1, shuffle = False)

In [ ]:
pred_gen = model.predict(predict_input_func)

predictions = list(pred_gen)
predictions
# predictions give the predicted house value 

In [ ]:
# now finally calculating root mean square error
final_preds = []

for pred in predictions:
    final_preds.append(pred["predictions"])
    

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, final_preds)** 0.5

In [ ]:
# you will go back and check medianHouseValue "mean" if mean and mean_squarred error are very close then model is predicting 
# very well. right now its not predicting well. 